# Gradyan Kontrolü

Bu çalışma dosyasında gradyan kontrolünü uygulamayı ve kullanmayı öğreneceksiniz.

Mobil ödemeleri küresel olarak kullanılabilir hale getirmek için çalışan bir ekibin parçasısınız ve sahtekarlığı (fraud) tespit etmek için bir derin öğrenme modeli oluşturmanız isteniyor - biri ödeme yaptığında, ödemenin sahte olup olmadığını, örneğin kullanıcının bir bilgisayar korsanı tarafından ele geçirildi.

Ancak geri yayılımın uygulanması oldukça zordur ve bazen hatalar olabilir. Bu kritik bir uygulama olduğundan, şirketinizin CEO'su geri yayılım uygulamanızın doğru olduğundan gerçekten emin olmak ister. CEO'nuz, "Bana geri yayılımınızın gerçekten işe yaradığına dair bir kanıt verin!" diyor. Bu güvenceyi vermek için "gradyan kontrolü" kullanacaksınız.

Haydi Yapalım şunu!

In [ ]:
# Packages
import numpy as np
#from testCases import *
#from gc_utils import sigmoid, relu, dictionary_to_vector, vector_to_dictionary, gradients_to_vector

In [ ]:
#@title Yardımcı Fonksiyonlar
import numpy as np

def sigmoid(x):
    """
    Compute the sigmoid of x
    Arguments:
    x -- A scalar or numpy array of any size.
    Return:
    s -- sigmoid(x)
    """
    s = 1/(1+np.exp(-x))
    return s

def relu(x):
    """
    Compute the relu of x
    Arguments:
    x -- A scalar or numpy array of any size.
    Return:
    s -- relu(x)
    """
    s = np.maximum(0,x)
    
    return s

def dictionary_to_vector(parameters):
    """
    Roll all our parameters dictionary into a single vector satisfying our specific required shape.
    """
    keys = []
    count = 0
    for key in ["W1", "b1", "W2", "b2", "W3", "b3"]:
        
        # flatten parameter
        new_vector = np.reshape(parameters[key], (-1,1))
        keys = keys + [key]*new_vector.shape[0]
        
        if count == 0:
            theta = new_vector
        else:
            theta = np.concatenate((theta, new_vector), axis=0)
        count = count + 1

    return theta, keys

def vector_to_dictionary(theta):
    """
    Unroll all our parameters dictionary from a single vector satisfying our specific required shape.
    """
    parameters = {}
    parameters["W1"] = theta[:20].reshape((5,4))
    parameters["b1"] = theta[20:25].reshape((5,1))
    parameters["W2"] = theta[25:40].reshape((3,5))
    parameters["b2"] = theta[40:43].reshape((3,1))
    parameters["W3"] = theta[43:46].reshape((1,3))
    parameters["b3"] = theta[46:47].reshape((1,1))

    return parameters

def gradients_to_vector(gradients):
    """
    Roll all our gradients dictionary into a single vector satisfying our specific required shape.
    """
    
    count = 0
    for key in ["dW1", "db1", "dW2", "db2", "dW3", "db3"]:
        # flatten parameter
        new_vector = np.reshape(gradients[key], (-1,1))
        
        if count == 0:
            theta = new_vector
        else:
            theta = np.concatenate((theta, new_vector), axis=0)
        count = count + 1

    return theta

In [ ]:
#@title Yardımcı Test Fonksiyonları
import numpy as np

def gradient_check_n_test_case(): 
    np.random.seed(1)
    x = np.random.randn(4,3)
    y = np.array([1, 1, 0])
    W1 = np.random.randn(5,4) 
    b1 = np.random.randn(5,1) 
    W2 = np.random.randn(3,5) 
    b2 = np.random.randn(3,1) 
    W3 = np.random.randn(1,3) 
    b3 = np.random.randn(1,1) 
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}

    
    return x, y, parameters

## 1) Gradyan kontrolü nasıl çalışır?

Geri yayılım $\frac{\partial J}{\partial \theta}$ gradyanlarını hesaplar, burada $\theta$ modelin parametrelerini gösterir. $J$, ileriye doğru yayılma ve sizin kayıp fonksiyonunuz kullanılarak hesaplanır.

İleri yayılımın uygulanması nispeten kolay olduğu için, bunu doğru yaptığınızdan eminsiniz ve dolayısıyla $J$ maliyetini doğru hesapladığınızdan neredeyse %100 eminsiniz. Böylece, $\frac{\partial J}{\partial \theta}$ hesaplama kodunu doğrulamak için kodunuzu $J$ hesaplamak için kullanabilirsiniz.

Bir türevin (veya gradyanın) tanımına tekrar bakalım:
$$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$

"$\displaystyle \lim_{\varepsilon \to 0}$" gösterimine aşina değilseniz, bu sadece "$\varepsilon$ gerçekten çok küçük olduğunda" demenin bir yoludur.

Aşağıdakileri biliyoruz:

- $\frac{\partial J}{\partial \theta}$, doğru hesaplama yaptığınızdan emin olmak istediğiniz şeydir.
- $J(\theta + \varepsilon)$ ve $J(\theta - \varepsilon)$'ı ($\theta$'ın gerçek bir sayı olması durumunda) hesaplayabilirsiniz, çünkü $J$ için uygulama doğrudur.


CEO'nuzu $\frac{\partial J}{\partial \theta}$ hesaplama kodunuzun doğru olduğuna ikna etmek için denklem (1) ve $\varepsilon$ için küçük bir değer kullanalım!

## 2) 1 boyutlu gradyan kontrolü

Bir 1B doğrusal fonksiyon $J(\theta) = \theta x$ düşünün. Model yalnızca tek bir gerçek değerli parametre $\theta$ içerir ve girdi olarak $x$ alır.

$J(.)$ ve onun türevini $\frac{\partial J}{\partial \theta}$ hesaplamak için kod uygulayacaksınız. Daha sonra $J$ için türev hesaplamanızın doğru olduğundan emin olmak için gradyan kontrolünü kullanacaksınız.

<img src="https://drive.google.com/uc?id=1NMOvE2Q2vklLB5G4Nsm7BGc3SNf3ApkR" style="width:600px;height:250px;" alt="1Dgrad_kiank" title="1Dgrad_kiank">
<caption><center> <u> **Şekil 1** </u>: **1B doğrusal model**<br> </center></caption>

Yukarıdaki şema, temel hesaplama adımlarını göstermektedir: Önce $x$ ile başlayın, ardından $J(x)$ ("ileri yayılım") işlevini değerlendirin. Ardından, $\frac{\partial J}{\partial \theta}$ ("geriye doğru yayılma") türevini hesaplayın.

**Alıştırma**: Bu basit işlev için "ileri yayılım" ve "geri yayılım" uygulayın. Yani, hem $J(.)$ ("ileri yayılma") hem de onun türevini $\theta$'a ("geri yayılma") göre iki ayrı fonksiyonda hesaplayın.

In [ ]:
# GRADED FUNCTION: forward_propagation

def forward_propagation(x, theta):
    """
    Implement the linear forward propagation (compute J) presented in Figure 1 (J(theta) = theta * x)
    
    Arguments:
    x -- a real-valued input
    theta -- our parameter, a real number as well
    
    Returns:
    J -- the value of function J, computed using the formula J(theta) = theta * x
    """
    
    ### START CODE HERE ### (approx. 1 line)
    J = np.dot(theta, x)
    ### END CODE HERE ###
    
    return J

In [ ]:
x, theta = 2, 4
J = forward_propagation(x, theta)
print ("J = " + str(J))

**Alıştırma**: Şimdi, Şekil 1'deki geriye doğru yayılma adımını (türevsel hesaplama) uygulayın. Yani, $J(\theta) = \theta x$'ın $\theta$'a göre türevini hesaplayın. Sizi hesap yapmaktan kurtarmak için $dtheta = \frac { \partial J }{ \partial \theta} = x$ elde etmelisiniz.

In [ ]:
# GRADED FUNCTION: backward_propagation

def backward_propagation(x, theta):
    """
    Computes the derivative of J with respect to theta (see Figure 1).
    
    Arguments:
    x -- a real-valued input
    theta -- our parameter, a real number as well
    
    Returns:
    dtheta -- the gradient of the cost with respect to theta
    """
    
    ### START CODE HERE ### (approx. 1 line)
    dtheta = x
    ### END CODE HERE ###
    
    return dtheta

In [ ]:
x, theta = 2, 4
dtheta = backward_propagation(x, theta)
print ("dtheta = " + str(dtheta))

**Alıştırma**: `backward_propagation()` fonksiyonunun $\frac{\partial J}{\partial \theta}$ gradyanını doğru bir şekilde hesapladığını göstermek için gradyan kontrolünü uygulayalım.

**Talimatlar**:
- İlk önce yukarıdaki formülü (1) ve küçük bir $\varepsilon$ değerini kullanarak "yaklaşık" derecesini hesaplayın. İzlenecek Adımlar:
  1. $\theta^{+} = \theta + \varepsilon$
  2. $\theta^{-} = \theta - \varepsilon$
  3. $J^{+} = J(\theta^{+})$
  4. $J^{-} = J(\theta^{-})$
  5. $gradapprox = \frac{J^{+} - J^{-}}{2  \varepsilon}$

- Ardından geriye doğru yayılmayı kullanarak gradyanı hesaplayın ve sonucu bir "grad" değişkeninde saklayın

- Son olarak, aşağıdaki formülü kullanarak "yaklaşık dereceli" ve "dereceli" arasındaki bağıl farkı hesaplayın:

$$ difference = \frac {\mid\mid grad - gradapprox \mid\mid_2}{\mid\mid grad \mid\mid_2 + \mid\mid gradapprox \mid\mid_2} \tag{2}$$

Bu formülü hesaplamak için 3 Adıma ihtiyacınız olacak:
  - 1'. np.linalg.norm(...) kullanarak payı hesaplayın
  - 2'. paydayı hesaplayın. np.linalg.norm(...)'u iki kez aramanız gerekecek.
  - 3'. onları böl.

- Bu fark küçükse (10$^{-7}$'dan az diyelim), gradyanınızı doğru hesapladığınızdan oldukça emin olabilirsiniz. Aksi takdirde, gradyan hesaplamasında bir hata olabilir.

In [ ]:
# GRADED FUNCTION: gradient_check

def gradient_check(x, theta, epsilon=1e-7):
    """
    Implement the backward propagation presented in Figure 1.
    
    Arguments:
    x -- a real-valued input
    theta -- our parameter, a real number as well
    epsilon -- tiny shift to the input to compute approximated gradient with formula(1)
    
    Returns:
    difference -- difference (2) between the approximated gradient and the backward propagation gradient
    """
    
    # Compute gradapprox using left side of formula (1). epsilon is small enough, you don't need to worry about the limit.
    ### START CODE HERE ### (approx. 5 lines)
    thetaplus = theta + epsilon                               # Step 1
    thetaminus = theta - epsilon                              # Step 2
    J_plus = forward_propagation(x, thetaplus)                # Step 3
    J_minus = forward_propagation(x, thetaminus)              # Step 4
    gradapprox = (J_plus - J_minus) / (2 * epsilon)           # Step 5
    ### END CODE HERE ###
    
    # Check if gradapprox is close enough to the output of backward_propagation()
    ### START CODE HERE ### (approx. 1 line)
    grad = backward_propagation(x, theta)
    ### END CODE HERE ###
    
    ### START CODE HERE ### (approx. 1 line)
    numerator = np.linalg.norm(grad - gradapprox)                      # Step 1'
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)    # Step 2'
    difference = numerator / denominator                               # Step 3'
    ### END CODE HERE ###
    
    if difference < 1e-7:
        print("The gradient is correct!")
    else:
        print("The gradient is wrong!")
    
    return difference

In [ ]:
x, theta = 2, 4
difference = gradient_check(x, theta)
print("difference = " + str(difference))

Tebrikler, fark 10$^{-7}$ eşiğinden daha küçük. Böylece, `backward_propagation()` içindeki gradyanı doğru hesapladığınızdan emin olabilirsiniz.

Şimdi, daha genel durumda, maliyet fonksiyonunuz $J$'ın birden fazla 1D girdisi var. Bir sinir ağını eğitirken, $\theta$ aslında birden çok $W^{[l]}$ matrisinden ve $b^{[l]}$ sapmalarından oluşur! Daha yüksek boyutlu girdilerle gradyan kontrolünün nasıl yapıldığını bilmek önemlidir. Haydi Yapalım şunu!

## 3) N boyutlu gradyan kontrolü

Aşağıdaki şekil, dolandırıcılık tespit modelinizin ileriye ve geriye doğru yayılmasını açıklamaktadır.

<img src="https://drive.google.com/uc?id=1uFZTCD9kNXn92G5JNzyZbiJJxxMu9Ggn" style="width:600px;height:400px;" alt="NDgrad_kiank" title="NDgrad_kiank">

<caption><center> <u> **Şekil 2** </u>: **derin sinir ağı**<br>*LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID*</center></caption>

İleriye yayılma ve geriye yayılma için uygulamalarınıza bakalım.

In [ ]:
def forward_propagation_n(X, Y, parameters):
    """
    Implements the forward propagation (and computes the cost) presented in Figure 3.
    
    Arguments:
    X -- training set for m examples
    Y -- labels for m examples 
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3":
                    W1 -- weight matrix of shape (5, 4)
                    b1 -- bias vector of shape (5, 1)
                    W2 -- weight matrix of shape (3, 5)
                    b2 -- bias vector of shape (3, 1)
                    W3 -- weight matrix of shape (1, 3)
                    b3 -- bias vector of shape (1, 1)
    
    Returns:
    cost -- the cost function (logistic cost for one example)
    """
    
    # retrieve parameters
    m = X.shape[1]
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]

    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)

    # Cost
    logprobs = np.multiply(-np.log(A3), Y) + np.multiply(-np.log(1 - A3), 1 - Y)
    cost = 1. / m * np.sum(logprobs)
    
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    return cost, cache

Şimdi, geriye doğru yayılmayı çalıştırın.

In [ ]:
def backward_propagation_n(X, Y, cache):
    """
    Implement the backward propagation presented in figure 2.
    
    Arguments:
    X -- input datapoint, of shape (input size, 1)
    Y -- true "label"
    cache -- cache output from forward_propagation_n()
    
    Returns:
    gradients -- A dictionary with the gradients of the cost with respect to each parameter, activation and pre-activation variables.
    """
    
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1. / m * np.dot(dZ3, A2.T)
    db3 = 1. / m * np.sum(dZ3, axis=1, keepdims=True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1. / m * np.dot(dZ2, A1.T) * 2  # Should not multiply by 2
    db2 = 1. / m * np.sum(dZ2, axis=1, keepdims=True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1. / m * np.dot(dZ1, X.T)
    db1 = 4. / m * np.sum(dZ1, axis=1, keepdims=True) # Should not multiply by 4
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

Sahtekarlık tespit test setinde bazı sonuçlar elde ettiniz ancak modelinizden %100 emin değilsiniz. Kimse mükemmel değildir! Degradelerinizin doğru olup olmadığını doğrulamak için degrade denetimi uygulayalım.

**Grandyan denetimi nasıl çalışır?**.

1) ve 2)'de olduğu gibi, "yaklaşık gradyan"ı geri yayılım tarafından hesaplanan gradyanla karşılaştırmak istiyorsunuz. Formül hala:

$$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$

Ancak, $\theta$ artık bir skaler değildir.  "parameters" adlı bir sözlüktür. Sizin için "`dictionary_to_vector()`" fonksiyonunu uyguladık.  "parameters" sözlüğünü, tüm parametreleri (W1, b1, W2, b2, W3, b3) vektörlere dönüştürerek ve birleştirerek elde edilen "değerler" adlı bir vektöre dönüştürür.

Ters işlev, "parameters" sözlüğünü geri veren "`vector_to_dictionary`" işlevidir.

<img src="https://drive.google.com/uc?id=1wndk8bJjkKW2Bl9EmeOWgXoP6WQGwT6f" style="width:600px;height:400px;" alt="dictionary_to_vector" title="dictionary_to_vector">

<caption><center> <u> **Şekil 3** </u>: **dictionary_to_vector() ve vector_to_dictionary()**<br> Bu fonksiyonlara gradyan_check_n() içinde ihtiyacınız olacak</center></caption>

Ayrıca "gradients" sözlüğünü degrades_to_vector() kullanarak bir "grad" vektörüne dönüştürdük. Bunun için endişelenmene gerek yok.

**Alıştırma**: gradient_check_n() uygulayın.

**Talimatlar**: Degrade kontrolünü uygulamanıza yardımcı olacak sözde kod burada.

num_parameters içindeki her i için:
- "J_plus[i]" hesaplamak için:
     1. $\theta^{+}$ öğesini `np.copy(parameters_values)` olarak ayarlayın
     2. $\theta^{+}_i$'ı $\theta^{+}_i + \varepsilon$ olarak ayarlayın
     3. `forward_propagation_n(x, y, vector_to_dictionary(`$\theta^{+}$ `))` kullanarak $J^{+}_i$ değerini hesaplayın.
- `J_minus[i]` hesaplamak için: $\theta^{-}$ ile aynı şeyi yapın
- $gradapprox[i] = \frac{J^{+}_i - J^{-}_i}{2 \varepsilon}$ hesaplayın

Böylece, gradapprox vektör elde edersiniz; burada gradapprox[i], `parameter_values[i]`ne göre gradyanın bir tahminidir. Şimdi bu gradyan vektörünü geri yayılımdan gradyan vektörüyle karşılaştırabilirsiniz. 1B durum için olduğu gibi (Adım 1', 2', 3'), hesaplayın:

$$ difference = \frac {\| grad - gradapprox \|_2}{\| grad \|_2 + \| gradapprox \|_2 } \tag{3}$$


In [ ]:
# GRADED FUNCTION: gradient_check_n

def gradient_check_n(parameters, gradients, X, Y, epsilon=1e-7):
    """
    Checks if backward_propagation_n computes correctly the gradient of the cost output by forward_propagation_n
    
    Arguments:
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3":
    grad -- output of backward_propagation_n, contains gradients of the cost with respect to the parameters. 
    x -- input datapoint, of shape (input size, 1)
    y -- true "label"
    epsilon -- tiny shift to the input to compute approximated gradient with formula(1)
    
    Returns:
    difference -- difference (2) between the approximated gradient and the backward propagation gradient
    """
    
    # Set-up variables
    parameters_values, _ = dictionary_to_vector(parameters)
    grad = gradients_to_vector(gradients)
    num_parameters = parameters_values.shape[0]
    J_plus = np.zeros((num_parameters, 1))
    J_minus = np.zeros((num_parameters, 1))
    gradapprox = np.zeros((num_parameters, 1))
    
    # Compute gradapprox
    for i in range(num_parameters):
        
        # Compute J_plus[i]. Inputs: "parameters_values, epsilon". Output = "J_plus[i]".
        # "_" is used because the function you have to outputs two parameters but we only care about the first one
        ### START CODE HERE ### (approx. 3 lines)
        thetaplus =  np.copy(parameters_values)                                       # Step 1
        thetaplus[i][0] = thetaplus[i][0] + epsilon                                   # Step 2
        J_plus[i], _ =  forward_propagation_n(X, Y, vector_to_dictionary(thetaplus))  # Step 3
        ### END CODE HERE ###
        
        # Compute J_minus[i]. Inputs: "parameters_values, epsilon". Output = "J_minus[i]".
        ### START CODE HERE ### (approx. 3 lines)
        thetaminus = np.copy(parameters_values)                                       # Step 1
        thetaminus[i][0] = thetaminus[i][0] - epsilon                                 # Step 2        
        J_minus[i], _ = forward_propagation_n(X, Y, vector_to_dictionary(thetaminus)) # Step 3
        ### END CODE HERE ###
        
        # Compute gradapprox[i]
        ### START CODE HERE ### (approx. 1 line)
        gradapprox[i] = (J_plus[i] - J_minus[i]) / (2 * epsilon)
        ### END CODE HERE ###
    
    # Compare gradapprox to backward propagation gradients by computing difference.
    ### START CODE HERE ### (approx. 1 line)
    numerator = np.linalg.norm(grad - gradapprox)                                     # Step 1'
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)                   # Step 2'
    difference = numerator / denominator                                              # Step 3'
    ### END CODE HERE ###

    if difference > 1e-7:
        print("There is a mistake in the backward propagation! difference = " + str(difference))
    else:
        print("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")
    
    return difference

In [ ]:
X, Y, parameters = gradient_check_n_test_case()

cost, cache = forward_propagation_n(X, Y, parameters)
gradients = backward_propagation_n(X, Y, cache)
difference = gradient_check_n(parameters, gradients, X, Y)

Size verdiğimiz `backward_propagation_n` kodunda hatalar var gibi görünüyor! Degrade kontrolünü uyguladığınız iyi oldu. `backward_propagation`a geri dönün ve hataları bulmaya/düzeltmeye çalışın *(İpucu: dW2 ve db1'i kontrol edin*). Düzelttiğinizi düşündüğünüzde degrade kontrolünü yeniden çalıştırın. Kodu değiştirirseniz, `backward_propagation_n()` tanımlayan hücreyi yeniden çalıştırmanız gerekeceğini unutmayın.

**Not**
- Gradyan Kontrolü yavaş! Gradyanı $\frac{\partial J}{\partial \theta} \approx  \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon}$ ile yaklaşık olarak hesaplamak hesaplama açısından maliyetlidir . Bu nedenle, eğitim sırasında her yinelemede gradyan kontrolü yapmıyoruz. Gradyanın doğru olup olmadığını kontrol etmek için sadece birkaç kez.

- Gradyan Kontrolü, en azından sunduğumuz gibi, bırakma ile çalışmaz. Backprop'unuzun doğru olduğundan emin olmak için genellikle gradyan kontrol algoritmasını bırakma olmadan çalıştırırsınız, ardından bırakma eklersiniz.

Tebrikler, dolandırıcılık tespiti için derin öğrenme modelinizin doğru çalıştığından emin olabilirsiniz! Bunu CEO'nuzu ikna etmek için bile kullanabilirsiniz. :)


<font color='blue'>
**Bu çalışma dosyasında hatırlamanız gerekenler**:

- Gradyan kontrolü, geri yayılımdan gelen gradyanlar ile gradyanın sayısal yaklaşımı (ileri yayılım kullanılarak hesaplanır) arasındaki yakınlığı doğrular.

- Gradyan kontrolü yavaştır, bu yüzden onu her eğitim yinelemesinde çalıştırmıyoruz. Genellikle yalnızca kodunuzun doğru olduğundan emin olmak için çalıştırır, ardından kapatır ve gerçek öğrenme süreci için backprop kullanırsınız.